In [1]:
import pandas as pd
from datetime import datetime
from ipywidgets import VBox, HBox, Button, Text, DatePicker, Checkbox, Dropdown, Label, TimePicker, Output
from ipywidgets import HTML as WidgetHTML
from IPython.display import HTML
from functools import partial

# File to store tasks
TASKS_FILE = "tasks.csv"

# Load existing tasks or create new
def load_tasks():
    try:
        return pd.read_csv(TASKS_FILE)
    except FileNotFoundError:
        return pd.DataFrame(columns=["Completed", "Task", "Company", "Deadline"])
    
# Save tasks
def save_tasks(df):
    df.to_csv(TASKS_FILE, index=False)
    
# Delete tasks 
def delete_task(b, idx):
    global tasks_df
    tasks_df = tasks_df.drop(index=idx).reset_index(drop=True)
    save_tasks(tasks_df)
    display_tasks()
    
# Complete tasks
def mark_complete(b, idx):
    global tasks_df
    tasks_df.at[idx, "Completed"] = True
    save_tasks(tasks_df)
    display_tasks()
    
# Create widgets for task entry
task_input = Text(description="Task:")
company_input = Text(description="Company:")
date_input = DatePicker(description="Date:")
time_input = Text(description="Time:", placeholder="e.g. 14:30")
add_button = Button(description="Add Task", button_style="success")
output = Output()

# Load and show task list
tasks_df = load_tasks()

# Controls for filtering and sorting
show_completed = True
sort_column = "Deadline"
sort_ascending = True

# Define toggle completion
def toggle_completion(change, idx):
   tasks_df.at[idx, "Completed"] = change["new"]
   save_tasks(tasks_df)
    
# Define toggle completion
def toggle_complete(b, idx):
    global tasks_df
    current_status = tasks_df.at[idx, "Completed"]
    tasks_df.at[idx, "Completed"] = not current_status
    save_tasks(tasks_df)
    display_tasks()

def display_tasks():
    global tasks_df
    output.clear_output()
    with output:
        if tasks_df.empty:
            print("No tasks yet.")
        else:
            df = tasks_df.copy()
            df["Deadline"] = pd.to_datetime(df["Deadline"], errors="coerce")

            # Apply filters
            if not show_completed:
                df = df[df["Completed"] == False]

            # Sort
            df = df.sort_values(by=sort_column, ascending=sort_ascending).reset_index(drop=True)

            # Header row with sorting buttons
            header = HBox([
                Button(description="Task", layout={"width": "250px"}, button_style=""),
                Button(description="Company", layout={"width": "150px"}, button_style=""),
                Button(description="Deadline", layout={"width": "140px"}, button_style=""),
                Label("Delete", layout={"width": "80px"}),
                Label("Toggle", layout={"width": "90px"})
            ])
            header.children[0].on_click(set_sort("Task"))
            header.children[1].on_click(set_sort("Company"))
            header.children[2].on_click(set_sort("Deadline"))

            rows = []
            for idx, row in df.iterrows():
                is_completed = row["Completed"]
                task_display = f"✓ {row['Task']}" if is_completed else row["Task"]

                deadline_dt = pd.to_datetime(row["Deadline"])
                deadline_str = deadline_dt.strftime("%m-%d %I:%M %p")
                is_overdue = not is_completed and deadline_dt < datetime.now()

                # Style if overdue
                task_style = "color: red; font-weight: bold;" if is_overdue else ""
                company_style = "color: red; font-weight: bold;" if is_overdue else ""
                deadline_style = "color: red; font-weight: bold;" if is_overdue else ""
        
                task_label = WidgetHTML(f"<div style='width:250px; {task_style}'>{task_display}</div>")
                company_label = WidgetHTML(f"<div style='width:150px; {company_style}'>{row['Company']}</div>")
                deadline_label = WidgetHTML(f"<div style='width:140px; {deadline_style}'>{deadline_str}</div>")

                delete_button = Button(description="Delete", button_style="danger", layout={"width": "70px"})
                delete_button.on_click(partial(delete_task, idx=tasks_df.index[tasks_df["Task"] == row["Task"]].tolist()[0]))

                toggle_button = Button(
                    description="Undo" if is_completed else "Complete",
                    button_style="warning" if is_completed else "success",
                    layout={"width": "80px"}
                )
                toggle_button.on_click(partial(toggle_complete, idx=tasks_df.index[tasks_df["Task"] == row["Task"]].tolist()[0]))

                row_box = HBox([
                    task_label,
                    company_label,
                    deadline_label,
                    delete_button,
                    toggle_button
                ])
                rows.append(row_box)

            display(VBox([header] + rows + [toggle_button_display]))
            
# Add task handler
def on_add_clicked(b):
    task = task_input.value
    company = company_input.value
    date = date_input.value
    time_str = time_input.value
    if not all([task, company, date, time_str]):
        print("Please fill in all fields.")
        return
    
    try:
        deadline = f"{date.strftime('%Y-%m-%d')} {time_str}"
        pd.to_datetime(deadline)  # Validate datetime format
        tasks_df.loc[len(tasks_df)] = [False, task, company, deadline]
        save_tasks(tasks_df)
        task_input.value = company_input.value = time_input.value = ''
        display_tasks()
    except Exception as e:
        print(f"Error with deadline format: {e}")

add_button.on_click(on_add_clicked)

# UI layout
display(VBox([
    Label("Add a Task:"),
    HBox([task_input, company_input]),
    HBox([date_input, time_input]),
    add_button,
    output
]))

# Toggle show/hide completed
def toggle_show_completed(b):
    global show_completed
    show_completed = not show_completed
    b.description = "Show All Tasks" if not show_completed else "Hide Completed Tasks"
    display_tasks()

toggle_button_display = Button(
    description="Hide Completed Tasks",
    button_style="info",
    layout={"margin": "10px 0px 10px 0px"}
)
toggle_button_display.on_click(toggle_show_completed)

# Sorting button handlers
def set_sort(col):
    def handler(b):
        global sort_column, sort_ascending
        if sort_column == col:
            sort_ascending = not sort_ascending  # toggle direction
        else:
            sort_column = col
            sort_ascending = True
        display_tasks()
    return handler


# Show initial tasks
display_tasks()

